# Budget Analysis using Python and Google Spreadsheets

## Settings / configuration

In [1]:
# the folder in which we put our exported bank transaction files
clerkai_folder = "~/Documents/Clerk.ai"

# the name of the google spreadsheet we use to annotate transactions
transactions_gsheet_export_title = "Clerk.ai - Transactions"

# custom columns that you want to annotate your transactions with (optional: can be left empty)
additional_transactions_editable_columns = []

## Import libraries and set up some global helpers

Note that you need to [set up a Google OAuth project or service account for yourself](https://gspread.readthedocs.io/en/latest/oauth2.html) to be able to import/export from/to Google Spreadsheets

In [2]:
# an authorized gspread client is required for gsheets import/export
import gspread
gsheets_client = gspread.oauth()

In [3]:
# import and init clerk.ai notebook helpers / functions
from clerkai.nb_helpers import init_notebook_and_return_helpers
helpers = init_notebook_and_return_helpers(clerkai_folder)
transactions = helpers["transactions"]
download_and_store_gsheets_edits = helpers["download_and_store_gsheets_edits"]
from clerkai.utils import export_to_gsheets
import pandas as pd
import numpy as np

In [4]:
# a general notebook helper function
from IPython.display import display
def display_full_df(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', -1):  # more options can be specified also
        display(df)

## Import annotations and edits that we have done in Google Spreadsheets (if any)

In [5]:
from gspread import SpreadsheetNotFound, WorksheetNotFound
from pandas.errors import EmptyDataError
try:
    download_and_store_gsheets_edits(gsheets_client, transactions_gsheet_export_title, "Transaction files", "transaction_files")
except SpreadsheetNotFound:
    print("Spreadsheet not found (it will be created later)")
except WorksheetNotFound:
    print("Worksheet not found (it will be created later)")
except EmptyDataError:
    print("Worksheet empty (it will be populated later)")

Creating '2020-05-31 1249 (54b3)/Transaction files.gsheets.Clerk.ai - Transactions.Transaction files.2020-05-31 125143281.xlsx'


In [6]:
from gspread import SpreadsheetNotFound, WorksheetNotFound
from pandas.errors import EmptyDataError
try:
    download_and_store_gsheets_edits(gsheets_client, transactions_gsheet_export_title, "Transactions", "transactions")
except SpreadsheetNotFound:
    print("Spreadsheet not found (it will be created later)")
except WorksheetNotFound:
    print("Worksheet not found (it will be created later)")
except EmptyDataError:
    print("Worksheet empty (it will be populated later)")

Creating '2020-05-31 1249 (54b3)/Transactions.gsheets.Clerk.ai - Transactions.Transactions.2020-05-31 125144412.xlsx'


## Scan transactions files

On the first run of the cell below, no transactions will be parsed, since necessary information about the files will be missing. Go to the exported Google Spreadsheet, select the sheet "Transaction Files" and fill the following columns:

* `Account provider` - what you call the bank or service that exported the file (eg "Bank of America")
* `Account`- what you call the account
* `Content type` - one of the content types listed [here](https://github.com/clerklabs/python-clerkai/blob/master/clerkai/transactions/parse.py). If none match, please submit an issue at https://github.com/clerklabs/python-clerkai/issues and request it (or create it and then submit a PR).

Then re-run all cells in this notebook again.

In [7]:
(parsed_transaction_files_df, transaction_files_df, unsuccessfully_parsed_transaction_files, successfully_parsed_transaction_files, all_transactions_df, parsed_transactions_df, transactions_df, transaction_files_editable_columns, transactions_editable_columns) = transactions(failfast=False,keep_unmerged_previous_edits=False,additional_transactions_editable_columns=additional_transactions_editable_columns)

print("")
print(".:: Parsing done")
print("Amount of non-ignored transaction files:", len(unsuccessfully_parsed_transaction_files)+len(successfully_parsed_transaction_files))
print("Unsuccessfully parsed files:", len(unsuccessfully_parsed_transaction_files))
print("Successfully parsed files:", len(successfully_parsed_transaction_files))
print("Transactions including duplicates:", len(all_transactions_df))
print("Transactions:", len(transactions_df))
print("")

# to see which files are not parsed - due to errors
if len(unsuccessfully_parsed_transaction_files) > 0:
    print("Some transaction files were not parsed:")
    display_full_df(unsuccessfully_parsed_transaction_files)
else:
    print("All transaction files parsed successfully")

Merging edits from 1 edit file(s) and Transaction files.xlsx into a new Transaction files.xlsx (ignoring currently parsed data)
Creating '2020-05-31 1249 (54b3)/Transaction files.xlsx'
Merging edits from 1 edit file(s) and Transactions.xlsx into a new Transactions.xlsx (ignoring currently parsed data)
0        0  00:00:00     0      0            NaN   NaN   
1        0  00:00:00     0      0            NaN   NaN   

   Income/Expense Category Currency  Real Date           Bank Date  ...  \
0                      NaN      EUR        NaN  2020-05-07 0:00:00  ...   
1                      NaN      EUR        NaN  2019-12-30 0:00:00  ...   

  History reference                 Source transaction file: File name  \
0              54b3  Personkonto FI12 3456 7890 1234 56 - 2020.05.3...   
1              54b3  Personkonto FI12 3456 7890 1234 56 - 2020.05.3...   

  Source transaction file: File path  \
0                     @/Transactions   
1                     @/Transactions   

   Source 

## (Optional) Run classifiers for automatic annotation of transactions

In [8]:
# run classifiers here

## Export results to Google Spreadsheets for manual annotations

In [9]:
gsheet_url = export_to_gsheets(gsheets_client, transaction_files_df, transactions_gsheet_export_title, "Transaction files", "transaction_files", create_if_not_exists=True, editable_columns=transaction_files_editable_columns)
print("List of transactions files exported to %s" % gsheet_url)

List of transactions files exported to https://docs.google.com/spreadsheets/d/1fMmCimo8g0BigpnNHU2Wq_z57lEDX-X5zcSjjX5m-p4


In [10]:
if len(transactions_df) > 0:
    gsheet_url = export_to_gsheets(gsheets_client, transactions_df, transactions_gsheet_export_title, "Transactions", "transactions", create_if_not_exists=True, editable_columns=transactions_editable_columns)
    print("Transactions exported to %s" % gsheet_url)

Transactions exported to https://docs.google.com/spreadsheets/d/1fMmCimo8g0BigpnNHU2Wq_z57lEDX-X5zcSjjX5m-p4


## How to manually annotate, re-import and re-run classifiers

Go to the Google Spreadsheet and annotate all editable columns (marked with a white background instead of grey), then re-run all cells in this notebook to import those changes back and re-run automatic classifiers.

You can also run this notebook via the command line (launching the notebook only if errors occurred):

```
./run.sh
```

# Visualizations and reports

In [11]:
# TODO: clerkai 0.3.0+ will populate the proper columns for pivot tables to make sense

In [12]:
if len(transactions_df) > 0:
    # Amounts by month and "Income/Expense Category"
    pivot_table = pd.pivot_table(
        transactions_df,
        values='Amount',
        index=["Month", "Income/Expense Category"],
        fill_value=0,
        columns=[],
        aggfunc=np.sum,
        margins=True
    ).round({'Amount': 0})
    display_full_df(pivot_table)

,,Amount
Month,Income/Expense Category,
All,,0.0
